In [2]:
import kagglehub
from pathlib import Path
import shutil

In [3]:
paths = [kagglehub.dataset_download("stefanoleone992/ea-sports-fc-24-complete-player-dataset"),
         kagglehub.dataset_download("nyagami/ea-sports-fc-25-database-ratings-and-stats"),
         kagglehub.dataset_download("hubertsidorowicz/football-players-stats-2024-2025")]

ValueError: Invalid Kaggle credentials file at C:\Users\ardao\.kaggle\kaggle.json

In [4]:
raw_dir = Path("../data/raw")
for path in paths:
    download_dir = Path(path)
    if not download_dir.exists():
        print(f"Download directory {download_dir} does not exist, skipping.")
        continue
    dataset_name = download_dir.name
    dataset_raw_dir = raw_dir / dataset_name
    dataset_raw_dir.mkdir(parents=True, exist_ok=True)
    for item in download_dir.iterdir():
        if item.is_file():
            dest = dataset_raw_dir / item.name
            shutil.copy2(item, dest)

NameError: name 'paths' is not defined

In [290]:
import pandas as pd
import numpy as np
from pathlib import Path
from fuzzywuzzy import fuzz, process

In [291]:
ratings_24_path = Path("../data/raw/4/male_players.csv")
ratings_25_path = Path("../data/raw/5/male_players.csv")

ratings_24_df = pd.read_csv(ratings_24_path)
ratings_25_df = pd.read_csv(ratings_25_path)

C:\Users\ardao\AppData\Local\Temp\ipykernel_28240\2060328440.py:4: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_24_df = pd.read_csv(ratings_24_path)


In [292]:
league_names_24 = ["La Liga", "Premier League", "Serie A", "Ligue 1", "Bundesliga"]
league_names_25 = ["LALIGA EA SPORTS", "Premier League", "Serie A Enilive", "Ligue 1 McDonald's", "Bundesliga"]

ratings_25_df = ratings_25_df[ratings_25_df['League'].isin(league_names_25)]

# SONRA isim dönüşümünü yap
def transform_25_league_names(league_name):
    mapping = {
        "LALIGA EA SPORTS": "La Liga",
        "Serie A Enilive": "Serie A",
        "Ligue 1 McDonald's": "Ligue 1"
    }
    return mapping.get(league_name, league_name)

ratings_25_df['League'] = ratings_25_df['League'].apply(transform_25_league_names)

ratings_24_df = ratings_24_df[ratings_24_df['league_name'].isin(league_names_24)]
ratings_24_df = ratings_24_df[ratings_24_df["fifa_version"]==24.0]

In [293]:
ratings_24_df.head()

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
5,188545,/player/188545/robert-lewandowski/240002,24.0,2.0,2023-09-22,R. Lewandowski,Robert Lewandowski,ST,90,90,...,66+3,66+3,66+3,65+3,61+3,61+3,61+3,61+3,61+3,19+3
6,192119,/player/192119/thibaut-courtois/240002,24.0,2.0,2023-09-22,T. Courtois,Thibaut Nicolas Marc Courtois,GK,90,90,...,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,88+2


In [294]:
ratings_24_df.shape

(3467, 109)

In [295]:
ratings_25_df.head(50)

,Unnamed: 0.1,Unnamed: 0,Rank,Name,OVR,PAC,SHO,PAS,DRI,DEF,...,Nation,League,Team,play style,url,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,0,0,1,Kylian Mbappé,91,97,90,80,92,36,...,France,La Liga,Real Madrid,"Quick Step+, Acrobatic, Finesse Shot, Flair, R...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
1,1,1,2,Rodri,91,66,80,86,84,87,...,Spain,Premier League,Manchester City,"Tiki Taka+, Aerial, Bruiser, Long Ball Pass, P...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
2,2,2,4,Erling Haaland,91,88,92,70,81,45,...,Norway,Premier League,Manchester City,"Acrobatic+, Bruiser, Power Header, Power Shot,...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
3,3,3,5,Jude Bellingham,90,80,87,83,88,78,...,England,La Liga,Real Madrid,"Relentless+, Flair, Intercept, Slide Tackle, T...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
4,4,4,7,Vini Jr.,90,95,84,81,91,29,...,Brazil,La Liga,Real Madrid,"Quick Step+, Chip Shot, Finesse Shot, First To...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
5,5,5,9,Kevin De Bruyne,90,67,87,94,87,65,...,Belgium,Premier League,Manchester City,"Incisive Pass+, Dead Ball, Long Ball Pass, Pin...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
6,6,6,11,Harry Kane,90,65,93,84,83,49,...,England,Bundesliga,FC Bayern München,"Finesse Shot+, First Touch, Incisive Pass, Lon...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
7,7,7,12,Martin Ødegaard,89,70,82,89,89,67,...,Norway,Premier League,Arsenal,"Incisive Pass+, First Touch, Flair, Pinged Pas...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
8,8,8,13,Gianluigi Donnarumma,89,90,84,73,90,52,...,Italy,Ligue 1,Paris SG,"Deflector+, Footwork",https://www.ea.com/games/ea-sports-fc/ratings/...,90.0,84.0,73.0,87.0,90.0
9,9,9,15,Alisson,89,86,85,85,89,56,...,Brazil,Premier League,Liverpool,"Deflector+, 1v1 Close Down, Far Throw",https://www.ea.com/games/ea-sports-fc/ratings/...,86.0,85.0,85.0,90.0,89.0


In [296]:
ratings_25_df.shape

(2612, 58)

In [297]:
def transform_name_to_short_format(name):
    if pd.isna(name):
        return name
    
    name_parts = str(name).strip().split()
    
    if len(name_parts) < 2:
        return name
    
    first_initial = name_parts[0][0].upper() + "."

    name = f"{first_initial} {' '.join(name_parts[1:])}"

    name = str(name).strip()
    return name
    
    # return f"{first_initial} {' '.join(name_parts[1:])}"



In [298]:
ratings_25_df['transformed_name'] = ratings_25_df['Name'].apply(transform_name_to_short_format)
ratings_25_df['transformed_name'].head(20)

0          K. Mbappé
1              Rodri
2         E. Haaland
3      J. Bellingham
4             V. Jr.
5       K. De Bruyne
6            H. Kane
7        M. Ødegaard
8      G. Donnarumma
9            Alisson
10       T. Courtois
11       L. Martínez
12       V. van Dijk
13     M. ter Stegen
14          M. Salah
15          P. Foden
17      A. Griezmann
18           R. Dias
19    R. Lewandowski
20       F. Valverde
Name: transformed_name, dtype: object

In [299]:
ratings_24_df["transformed_name"] = ratings_24_df["short_name"].apply(transform_name_to_short_format)
ratings_24_df['transformed_name'].head(20)

0          K. Mbappé
1         E. Haaland
2       K. De Bruyne
5     R. Lewandowski
6        T. Courtois
7            H. Kane
8             V. Jr.
9            Alisson
10             Rodri
11           R. Dias
13     M. ter Stegen
14          Casemiro
15       V. van Dijk
16          M. Salah
17       F. Valverde
18        V. Osimhen
19           Ederson
20        J. Kimmich
21      A. Griezmann
22          J. Oblak
Name: transformed_name, dtype: object

In [300]:
def handle_jr(name):
    if pd.isna(name):
        return name
    
    name = str(name).strip()
    # print(f"Original: '{name}', Length: {len(name)}")
    
    if name.endswith(" Jr."):
        result = name[:-4].strip() + " Junior"
        print(f"Matched ' Jr.' - Result: '{result}'")
        return result
    elif name.endswith(" Jr"):
        result = name[:-3].strip() + " Junior"
        print(f"Matched ' Jr' - Result: '{result}'")
        return result
    else:
        print(f"No match found for: '{name}'")

    name = str(name).strip()
    return name


In [301]:
# # Test etmek için bu kodu ekleyin
# def handle_jr_debug(name):
#     if pd.isna(name):
#         return name
    
#     name = str(name).strip()
#     print(f"Original: '{name}', Length: {len(name)}")
    
#     if name.endswith(" Jr."):
#         result = name[:-4].strip() + " Junior"
#         print(f"Matched ' Jr.' - Result: '{result}'")
#         return result
#     elif name.endswith(" Jr"):
#         result = name[:-3].strip() + " Junior"
#         print(f"Matched ' Jr' - Result: '{result}'")
#         return result
#     else:
#         print(f"No match found for: '{name}'")

#     return name

# # Test edin:
# test_names = ["Vinicius Jr.", "Vinicius Jr", "Vinícius Jr.", "Vinícius Junior"]
# for test_name in test_names:
#     print(f"Testing: {test_name}")
#     result = handle_jr_debug(test_name)
#     print(f"Result: {result}\n")

In [302]:
ratings_24_df['transformed_name'] = ratings_24_df['transformed_name'].apply(handle_jr)
ratings_25_df['transformed_name'] = ratings_25_df['transformed_name'].apply(handle_jr)

No match found for: 'K. Mbappé'
No match found for: 'E. Haaland'
No match found for: 'K. De Bruyne'
No match found for: 'R. Lewandowski'
No match found for: 'T. Courtois'
No match found for: 'H. Kane'
Matched ' Jr.' - Result: 'V. Junior'
No match found for: 'Alisson'
No match found for: 'Rodri'
No match found for: 'R. Dias'
No match found for: 'M. ter Stegen'
No match found for: 'Casemiro'
No match found for: 'V. van Dijk'
No match found for: 'M. Salah'
No match found for: 'F. Valverde'
No match found for: 'V. Osimhen'
No match found for: 'Ederson'
No match found for: 'J. Kimmich'
No match found for: 'A. Griezmann'
No match found for: 'J. Oblak'
No match found for: 'B. Fernandes'
No match found for: 'B. Silva'
No match found for: 'M. Ødegaard'
No match found for: 'F. de Jong'
No match found for: 'G. Donnarumma'
No match found for: 'L. Martínez'
No match found for: 'G. Kobel'
No match found for: 'M. Maignan'
No match found for: 'Marquinhos'
No match found for: 'M. Neuer'
No match found 

In [303]:
print("shapes are:", ratings_24_df.shape, ratings_25_df.shape)

shapes are: (3467, 110) (2612, 59)


In [304]:
ratings_24_df["transformed_name"].head(40)


0            K. Mbappé
1           E. Haaland
2         K. De Bruyne
5       R. Lewandowski
6          T. Courtois
7              H. Kane
8            V. Junior
9              Alisson
10               Rodri
11             R. Dias
13       M. ter Stegen
14            Casemiro
15         V. van Dijk
16            M. Salah
17         F. Valverde
18          V. Osimhen
19             Ederson
20          J. Kimmich
21        A. Griezmann
22            J. Oblak
23        B. Fernandes
24            B. Silva
25         M. Ødegaard
26          F. de Jong
27       G. Donnarumma
28         L. Martínez
29            G. Kobel
30          M. Maignan
31          Marquinhos
32            M. Neuer
33           L. Modrić
34              H. Son
35          J. Musiala
36    K. Kvaratskhelia
37               Pedri
38             R. Leão
39       J. Bellingham
40           R. Araujo
41             B. Saka
42          M. de Ligt
Name: transformed_name, dtype: object

In [305]:
ratings_25_df["transformed_name"].head(50)

0           K. Mbappé
1               Rodri
2          E. Haaland
3       J. Bellingham
4           V. Junior
5        K. De Bruyne
6             H. Kane
7         M. Ødegaard
8       G. Donnarumma
9             Alisson
10        T. Courtois
11        L. Martínez
12        V. van Dijk
13      M. ter Stegen
14           M. Salah
15           P. Foden
17       A. Griezmann
18            R. Dias
19     R. Lewandowski
20        F. Valverde
21            Ederson
22           B. Silva
23           J. Oblak
24         A. Rüdiger
25           F. Wirtz
26           G. Kobel
27          W. Saliba
28         V. Osimhen
29         J. Musiala
30            B. Saka
31          P. Dybala
32       T. Hernández
34            D. Rice
35         F. de Jong
36         M. Maignan
37       B. Fernandes
38         H. Min Son
39         Marquinhos
40         A. Bastoni
41        İ. Gündoğan
42        E. Martínez
43         N. Barella
44          Y. Sommer
46            Gabriel
47           G. Xhaka
48        

In [306]:
names_25 = ratings_25_df['transformed_name'].dropna()
names_24 = ratings_24_df['transformed_name'].dropna()
threshold = 0.80

# # 1. Find exact matches first
# exact_matches = set(names_25) & set(names_24)
# matches = []

# for name_25 in exact_matches:
#     matches.append({
#         'name_25': name_25,
#         'name_24': name_25,
#         'similarity_score': 100, 
#         'original_name_25': ratings_25_df[ratings_25_df['transformed_name'] == name_25]['Name'].iloc[0] if 'Name' in ratings_25_df.columns else name_25
#     })

In [307]:
vitinhas = names_25[names_25 == "Vitinha"]
vitinhas

74      Vitinha
1380    Vitinha
Name: transformed_name, dtype: object

In [308]:
from collections import defaultdict
from tqdm import tqdm

# Step 1: Duplicate isimleri tespit et
def find_duplicates(df, name_col, original_col):
    name_groups = defaultdict(list)
    for idx, row in df.iterrows():
        transformed_name = row[name_col]
        if pd.notna(transformed_name):
            name_groups[transformed_name].append({
                'index': idx,
                'original_name': row[original_col],
                'transformed_name': transformed_name
            })
    
    duplicates = {k: v for k, v in name_groups.items() if len(v) > 1}
    unique_names = {k: v[0] for k, v in name_groups.items() if len(v) == 1}
    
    return duplicates, unique_names

# FIFA 24 ve 25 için duplicate isimleri bul
duplicates_24, unique_24 = find_duplicates(ratings_24_df, 'transformed_name', 'short_name')
duplicates_25, unique_25 = find_duplicates(ratings_25_df, 'transformed_name', 'Name')

print(f"FIFA 24 - Duplicates: {len(duplicates_24)}, Unique: {len(unique_24)}")
print(f"FIFA 25 - Duplicates: {len(duplicates_25)}, Unique: {len(unique_25)}")

# Duplicate örnekleri göster
print("\nFIFA 24 Duplicate örnekleri:")
for name, players in list(duplicates_24.items())[:]:
    print(f"{name}: {[p['original_name'] for p in players]}")

print("\nFIFA 25 Duplicate örnekleri:")
for name, players in list(duplicates_25.items())[:]:
    print(f"{name}: {[p['original_name'] for p in players]}")

FIFA 24 - Duplicates: 54, Unique: 3351
FIFA 25 - Duplicates: 30, Unique: 2552

FIFA 24 Duplicate örnekleri:
V. Junior: ['Vini Jr.', 'Vasquão Junior']
Rodri: ['Rodri', 'Rodri']
L. Martínez: ['L. Martínez', 'L. Martínez']
Marquinhos: ['Marquinhos', 'Marquinhos']
A. Onana: ['A. Onana', 'A. Onana']
L. Hernández: ['L. Hernández', 'Luís Hernández']
A. Davies: ['A. Davies', 'A. Davies']
L. Pellegrini: ['L. Pellegrini', 'L. Pellegrini']
D. García: ['David García', 'Dani García']
Fernando: ['Fernando', 'Fernando', 'Fernando']
F. Mendy: ['F. Mendy', 'F. Mendy']
A. Sánchez: ['A. Sánchez', 'Antonio Sánchez']
Danilo: ['Danilo', 'Danilo', 'Danilo']
J. Álvarez: ['J. Álvarez', 'Joseda Álvarez']
Vitinha: ['Vitinha', 'Vitinha']
N. Williams: ['Nico Williams', 'N. Williams']
D. López: ['David López', 'Diego López']
M. Gómez: ['Moi Gómez', 'M. Gómez', 'M. Gómez']
J. Sá: ['José Sá', 'Joshuando Sá']
M. Camara: ['M. Camara', 'M. Camara']
M. Cunha: ['Matheus Cunha', 'Mardial Cunha']
M. Berisha: ['M. Berisha', 

In [309]:
def match_players_improved(unique_24, unique_25, duplicates_24, duplicates_25, threshold=0.80, age_diff_threshold=2):
    matches = []
    
    # Step 1: Unique isimleri eşleştir (exact match + age check)
    exact_matches = set(unique_24.keys()) & set(unique_25.keys())
    
    for name in exact_matches:
        player_24 = unique_24[name]
        player_25 = unique_25[name]
        
        # Yaş kontrolü - FIFA 24'ten 25'e 1 yıl geçmiş olmalı
        # Yani FIFA 25'teki yaş, FIFA 24'teki yaştan 0-2 fazla olmalı
        age_24 = ratings_24_df[ratings_24_df['short_name'] == player_24['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == player_24['original_name']]) > 0 else None
        age_25 = ratings_25_df[ratings_25_df['Name'] == player_25['original_name']]['Age'].iloc[0] if len(ratings_25_df[ratings_25_df['Name'] == player_25['original_name']]) > 0 else None
        
        # Yaş verisi varsa kontrol et
        if age_24 is not None and age_25 is not None and pd.notna(age_24) and pd.notna(age_25):
            age_difference = age_25 - age_24
            if age_difference < 0 or age_difference > age_diff_threshold:
                print(f"Age mismatch for {name}: FIFA 24 age={age_24}, FIFA 25 age={age_25}, diff={age_difference}")
                continue
        
        matches.append({
            'name_25': name,
            'name_24': name,
            'similarity_score': 100,
            'original_name_25': player_25['original_name'],
            'original_name_24': player_24['original_name'],
            'match_type': 'unique_exact',
            'age_24': age_24,
            'age_25': age_25,
            'age_diff': age_25 - age_24 if (age_24 is not None and age_25 is not None) else None
        })
    
    print(f"Unique exact matches (with age check): {len([m for m in matches if m['match_type'] == 'unique_exact'])}")
    
    # Step 2: Duplicate vs Duplicate eşleştirmeleri (fuzzy match + age check)
    common_duplicate_names = set(duplicates_24.keys()) & set(duplicates_25.keys())
    
    for name in common_duplicate_names:
        candidates_24 = duplicates_24[name]
        candidates_25 = duplicates_25[name]
        
        # Her 25 oyuncusu için en iyi 24 eşleşmesini bul (fuzzy match + age check)
        used_24_indices = set()
        
        for candidate_25 in candidates_25:
            best_match = None
            best_score = 0
            best_idx = None
            
            # FIFA 25 oyuncusunun yaşını al
            age_25 = ratings_25_df[ratings_25_df['Name'] == candidate_25['original_name']]['Age'].iloc[0] if len(ratings_25_df[ratings_25_df['Name'] == candidate_25['original_name']]) > 0 else None
            
            for i, candidate_24 in enumerate(candidates_24):
                if i in used_24_indices:
                    continue
                
                # FIFA 24 oyuncusunun yaşını al
                age_24 = ratings_24_df[ratings_24_df['short_name'] == candidate_24['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == candidate_24['original_name']]) > 0 else None
                
                # Yaş kontrolü
                if age_24 is not None and age_25 is not None and pd.notna(age_24) and pd.notna(age_25):
                    age_difference = age_25 - age_24
                    if age_difference < 0 or age_difference > age_diff_threshold:
                        continue
                    
                # Fuzzy match
                score = fuzz.ratio(candidate_25['original_name'], candidate_24['original_name'])
                if score > best_score:
                    best_score = score
                    best_match = candidate_24
                    best_idx = i
            
            if best_match and best_score >= threshold * 100:
                age_24 = ratings_24_df[ratings_24_df['short_name'] == best_match['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == best_match['original_name']]) > 0 else None
                matches.append({
                    'name_25': name,
                    'name_24': name,
                    'similarity_score': best_score,
                    'original_name_25': candidate_25['original_name'],
                    'original_name_24': best_match['original_name'],
                    'match_type': 'duplicate_vs_duplicate',
                    'age_24': age_24,
                    'age_25': age_25,
                    'age_diff': age_25 - age_24 if (age_24 is not None and age_25 is not None) else None
                })
                used_24_indices.add(best_idx)
    
    # Step 3: Unique vs Duplicate - exact match + age check
    # 25'te unique, 24'te duplicate olan durumlar
    for name_25 in unique_25:
        if name_25 not in [m['name_25'] for m in matches] and name_25 in duplicates_24:
            player_25 = unique_25[name_25]
            candidates_24 = duplicates_24[name_25]
            
            # FIFA 25 oyuncusunun yaşını al
            age_25 = ratings_25_df[ratings_25_df['Name'] == player_25['original_name']]['Age'].iloc[0] if len(ratings_25_df[ratings_25_df['Name'] == player_25['original_name']]) > 0 else None
            
            # Exact match + age check
            exact_match = None
            for candidate in candidates_24:
                if player_25['original_name'] == candidate['original_name']:
                    # Yaş kontrolü
                    age_24 = ratings_24_df[ratings_24_df['short_name'] == candidate['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == candidate['original_name']]) > 0 else None
                    
                    if age_24 is not None and age_25 is not None and pd.notna(age_24) and pd.notna(age_25):
                        age_difference = age_25 - age_24
                        if age_difference < 0 or age_difference > age_diff_threshold:
                            continue
                    
                    exact_match = candidate
                    break
            
            if exact_match:
                age_24 = ratings_24_df[ratings_24_df['short_name'] == exact_match['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == exact_match['original_name']]) > 0 else None
                matches.append({
                    'name_25': name_25,
                    'name_24': name_25,
                    'similarity_score': 100,
                    'original_name_25': player_25['original_name'],
                    'original_name_24': exact_match['original_name'],
                    'match_type': 'unique_vs_duplicate_exact',
                    'age_24': age_24,
                    'age_25': age_25,
                    'age_diff': age_25 - age_24 if (age_24 is not None and age_25 is not None) else None
                })
    
    # 24'te unique, 25'te duplicate olan durumlar
    for name_24 in unique_24:
        if name_24 not in [m['name_24'] for m in matches] and name_24 in duplicates_25:
            player_24 = unique_24[name_24]
            candidates_25 = duplicates_25[name_24]
            
            # FIFA 24 oyuncusunun yaşını al
            age_24 = ratings_24_df[ratings_24_df['short_name'] == player_24['original_name']]['age'].iloc[0] if len(ratings_24_df[ratings_24_df['short_name'] == player_24['original_name']]) > 0 else None
            
            # Exact match + age check
            exact_match = None
            for candidate in candidates_25:
                if player_24['original_name'] == candidate['original_name']:
                    # Yaş kontrolü
                    age_25 = ratings_25_df[ratings_25_df['Name'] == candidate['original_name']]['Age'].iloc[0] if len(ratings_25_df[ratings_25_df['Name'] == candidate['original_name']]) > 0 else None
                    
                    if age_24 is not None and age_25 is not None and pd.notna(age_24) and pd.notna(age_25):
                        age_difference = age_25 - age_24
                        if age_difference < 0 or age_difference > age_diff_threshold:
                            continue
                    
                    exact_match = candidate
                    break
            
            if exact_match:
                age_25 = ratings_25_df[ratings_25_df['Name'] == exact_match['original_name']]['Age'].iloc[0] if len(ratings_25_df[ratings_25_df['Name'] == exact_match['original_name']]) > 0 else None
                matches.append({
                    'name_25': name_24,
                    'name_24': name_24,
                    'similarity_score': 100,
                    'original_name_25': exact_match['original_name'],
                    'original_name_24': player_24['original_name'],
                    'match_type': 'duplicate_vs_unique_exact',
                    'age_24': age_24,
                    'age_25': age_25,
                    'age_diff': age_25 - age_24 if (age_24 is not None and age_25 is not None) else None
                })
    
    return matches

# Yeni eşleştirmeyi çalıştır
matches = match_players_improved(unique_24, unique_25, duplicates_24, duplicates_25)
matches_df = pd.DataFrame(matches)
matches_df = matches_df.sort_values('similarity_score', ascending=False)

print(f"\nTotal matches found: {len(matches_df)}")
print(f"Match types distribution:")
print(matches_df['match_type'].value_counts())
print("\nFirst 10 matches:")
print(matches_df[['original_name_25', 'original_name_24', 'similarity_score', 'match_type', 'age_24', 'age_25', 'age_diff']].head(10))

# Yaş farkı olan eşleştirmeleri kontrol et
age_issues = matches_df[matches_df['age_diff'] > 2]
print(f"\nMatches with age difference > 2:")
print(age_issues[['original_name_24', 'original_name_25', 'age_24', 'age_25', 'age_diff']])

# André Silva'yı kontrol et
andre_silva_matches = matches_df[
    (matches_df['original_name_24'].str.contains('André Silva', case=False, na=False)) |
    (matches_df['original_name_25'].str.contains('André Silva', case=False, na=False))
]
print(f"\nAndré Silva matches:")
print(andre_silva_matches[['original_name_24', 'original_name_25', 'similarity_score', 'match_type', 'age_24', 'age_25', 'age_diff']])

Age mismatch for Cédric: FIFA 24 age=31, FIFA 25 age=22, diff=-9
Age mismatch for S. Mitrović: FIFA 24 age=33, FIFA 25 age=22, diff=-11
Age mismatch for I. Koné: FIFA 24 age=24, FIFA 25 age=22, diff=-2
Age mismatch for A. Bamba: FIFA 24 age=29, FIFA 25 age=34, diff=5
Unique exact matches (with age check): 1728

Total matches found: 1764
Match types distribution:
match_type
unique_exact                 1728
duplicate_vs_duplicate         19
unique_vs_duplicate_exact      12
duplicate_vs_unique_exact       5
Name: count, dtype: int64

First 10 matches:
            original_name_25     original_name_24  similarity_score  \
1763          Juanlu Sánchez       Juanlu Sánchez               100   
0             Miguel Almirón           M. Almirón               100   
1                Reece James             R. James               100   
2               Reece Oxford            R. Oxford               100   
1724           Marvin Senaya            M. Senaya               100   
1725             

In [310]:
# Merged dataframe oluştur
def create_merged_dataframe_improved(matches_df, ratings_24_df, ratings_25_df, column_mapping):
    merged_data = []
    
    for _, match_row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Creating merged dataframe"):
        # Find exact player records by original name
        player_24 = ratings_24_df[ratings_24_df['short_name'] == match_row['original_name_24']]
        player_25 = ratings_25_df[ratings_25_df['Name'] == match_row['original_name_25']]
        
        if player_24.empty or player_25.empty:
            print(f"Warning: Could not find player - 24: {match_row['original_name_24']}, 25: {match_row['original_name_25']}")
            continue
            
        # Take first match if multiple (shouldn't happen with proper filtering)
        player_24 = player_24.iloc[0]
        player_25 = player_25.iloc[0]
            
        # Create merged row
        merged_row = {
            'matched_name': match_row['name_24'],
            'name_24': match_row['original_name_24'],
            'name_25': match_row['original_name_25'],
            'similarity_score': match_row['similarity_score'],
            'match_type': match_row['match_type']
        }
        
        # Add mapped columns
        for col24, col25 in column_mapping.items():
            if col24 in player_24.index and pd.notna(player_24[col24]):
                merged_row[f"{col24}_24"] = player_24[col24]
            if col25 in player_25.index and pd.notna(player_25[col25]):
                merged_row[f"{col24}_25"] = player_25[col25]
                
        merged_data.append(merged_row)
    
    return pd.DataFrame(merged_data)

# Merged dataframe oluştur
merged_df = create_merged_dataframe_improved(matches_df, ratings_24_df, ratings_25_df, column_mapping)

print(f"Merged dataframe shape: {merged_df.shape}")
print(f"Columns: {len(merged_df.columns)}")
print("\nFirst few rows:")
print(merged_df[['name_24', 'name_25', 'similarity_score', 'match_type']].head())

Creating merged dataframe: 100%|██████████| 1764/1764 [00:02<00:00, 881.51it/s]

Merged dataframe shape: (1764, 109)
Columns: 109

First few rows:
          name_24         name_25  similarity_score                 match_type
0  Juanlu Sánchez  Juanlu Sánchez               100  duplicate_vs_unique_exact
1      M. Almirón  Miguel Almirón               100               unique_exact
2        R. James     Reece James               100               unique_exact
3       R. Oxford    Reece Oxford               100               unique_exact
4       M. Senaya   Marvin Senaya               100               unique_exact


In [311]:
# list(exact_matches)[:5]

In [312]:
# len(exact_matches)

In [313]:
# names_25_remaining = set(names_25) - exact_matches
# names_24_remaining = set(names_24) - exact_matches

In [314]:
# len(names_24_remaining)

In [315]:
# len(names_25_remaining)

In [316]:
# from tqdm import tqdm

# for name_25 in tqdm(names_25_remaining, desc="Fuzzy matching"):
#     best_match = process.extractOne(name_25, list(names_24_remaining), scorer=fuzz.ratio)
#     if best_match and best_match[1] >= threshold * 100:
#         matches.append({
#             'name_25': name_25,
#             'name_24': best_match[0],
#             'similarity_score': best_match[1],
#             'original_name_25': ratings_25_df[ratings_25_df['transformed_name'] == name_25]['Name'].iloc[0] if 'Name' in ratings_25_df.columns else name_25
#         })

# matches_df = pd.DataFrame(matches)
# matches_df = matches_df.sort_values('similarity_score', ascending=False)

# print(f"Found {len(matches_df)} matches (exact + fuzzy) above threshold {threshold}")
# print(matches_df.head(20))

In [317]:
column_mapping = {
    # Player Identity
    'short_name': 'Name',
    'age': 'Age', 
    'nationality_name': 'Nation',
    'club_name': 'Team',
    'league_name': 'League',
    'player_positions': 'Position',
    'preferred_foot': 'Preferred foot',
    'weak_foot': 'Weak foot',
    'skill_moves': 'Skill moves',
    'height_cm': 'Height',
    'weight_kg': 'Weight',
    
    # Main Stats
    'overall': 'OVR',
    'pace': 'PAC',
    'shooting': 'SHO',
    'passing': 'PAS',
    'dribbling': 'DRI',
    'defending': 'DEF',
    'physic': 'PHY',
    
    # Detailed Stats
    'movement_acceleration': 'Acceleration',
    'movement_sprint_speed': 'Sprint Speed',
    'mentality_positioning': 'Positioning',
    'attacking_finishing': 'Finishing',
    'power_shot_power': 'Shot Power',
    'power_long_shots': 'Long Shots',
    'attacking_volleys': 'Volleys',
    'mentality_penalties': 'Penalties',
    'mentality_vision': 'Vision',
    'attacking_crossing': 'Crossing',
    'skill_fk_accuracy': 'Free Kick Accuracy',
    'attacking_short_passing': 'Short Passing',
    'skill_long_passing': 'Long Passing',
    'skill_curve': 'Curve',
    'skill_dribbling': 'Dribbling',
    'movement_agility': 'Agility',
    'movement_balance': 'Balance',
    'movement_reactions': 'Reactions',
    'skill_ball_control': 'Ball Control',
    'mentality_composure': 'Composure',
    'mentality_interceptions': 'Interceptions',
    'attacking_heading_accuracy': 'Heading Accuracy',
    'defending_marking_awareness': 'Def Awareness',
    'defending_standing_tackle': 'Standing Tackle',
    'defending_sliding_tackle': 'Sliding Tackle',
    'power_jumping': 'Jumping',
    'power_stamina': 'Stamina',
    'power_strength': 'Strength',
    'mentality_aggression': 'Aggression',
    
    # Goalkeeper Stats
    'goalkeeping_diving': 'GK Diving',
    'goalkeeping_handling': 'GK Handling',
    'goalkeeping_kicking': 'GK Kicking',
    'goalkeeping_positioning': 'GK Positioning',
    'goalkeeping_reflexes': 'GK Reflexes'
}

In [318]:
# fifa24_dict = {}
# for _, row in ratings_24_df.iterrows():
#     short_name = row['short_name']
#     if pd.notna(short_name):
#         fifa24_dict[short_name] = row.to_dict()

# fifa25_dict = {}
# for _, row in ratings_25_df.iterrows():
#     transformed_name = row['transformed_name']
#     if pd.notna(transformed_name):
#         fifa25_dict[transformed_name] = row.to_dict()

# # Step 3: Build merged dataframe
# merged_data = []

# for _, match_row in matches_df.iterrows():
#     name_24 = match_row['name_24']
#     name_25 = match_row['name_25']
    
#     # Get player data from both datasets
#     player_24 = fifa24_dict.get(name_24, {})
#     player_25 = fifa25_dict.get(name_25, {})
    
#     # Create merged row
#     merged_row = {
#         'matched_name': name_24,
#         'name_24': name_24,
#         'name_25': name_25,
#         'similarity_score': match_row['similarity_score'],
#         'original_name_25': match_row['original_name_25']
#     }
    
#     # Add mapped columns - create separate _24 and _25 columns
#     for col24, col25 in column_mapping.items():
#         if col24 in player_24:
#             merged_row[f"{col24}_24"] = player_24[col24]
#         if col25 in player_25:
#             merged_row[f"{col24}_25"] = player_25[col25]
            
#     merged_data.append(merged_row)

# merged_df = pd.DataFrame(merged_data)

In [319]:
merged_df.head()

,matched_name,name_24,name_25,similarity_score,match_type,short_name_24,short_name_25,age_24,age_25,nationality_name_24,...,goalkeeping_diving_24,goalkeeping_handling_24,goalkeeping_kicking_24,goalkeeping_positioning_24,goalkeeping_reflexes_24,goalkeeping_diving_25,goalkeeping_handling_25,goalkeeping_kicking_25,goalkeeping_positioning_25,goalkeeping_reflexes_25
0,J. Sánchez,Juanlu Sánchez,Juanlu Sánchez,100,duplicate_vs_unique_exact,Juanlu Sánchez,Juanlu Sánchez,19,21,Spain,...,15,12,6,12,13,NaN,NaN,NaN,NaN,NaN
1,M. Almirón,M. Almirón,Miguel Almirón,100,unique_exact,M. Almirón,Miguel Almirón,29,30,Paraguay,...,6,9,13,13,12,NaN,NaN,NaN,NaN,NaN
2,R. James,R. James,Reece James,100,unique_exact,R. James,Reece James,23,24,England,...,8,12,7,13,11,NaN,NaN,NaN,NaN,NaN
3,R. Oxford,R. Oxford,Reece Oxford,100,unique_exact,R. Oxford,Reece Oxford,24,25,England,...,10,11,7,9,14,NaN,NaN,NaN,NaN,NaN
4,M. Senaya,M. Senaya,Marvin Senaya,100,unique_exact,M. Senaya,Marvin Senaya,22,23,France,...,9,11,10,7,12,NaN,NaN,NaN,NaN,NaN


In [320]:
merged_df.shape

(1764, 109)

In [321]:
merged_df.tail(50)

,matched_name,name_24,name_25,similarity_score,match_type,short_name_24,short_name_25,age_24,age_25,nationality_name_24,...,goalkeeping_diving_24,goalkeeping_handling_24,goalkeeping_kicking_24,goalkeeping_positioning_24,goalkeeping_reflexes_24,goalkeeping_diving_25,goalkeeping_handling_25,goalkeeping_kicking_25,goalkeeping_positioning_25,goalkeeping_reflexes_25
1714,Aarón,Aarón,Aarón,100,unique_vs_duplicate_exact,Aarón,Aarón,26,27,Spain,...,11,14,10,12,12,NaN,NaN,NaN,NaN,NaN
1715,Danilo,Danilo,Danilo,100,duplicate_vs_duplicate,Danilo,Danilo,31,33,Brazil,...,14,5,15,14,5,NaN,NaN,NaN,NaN,NaN
1716,R. García,Rubén García,Rubén García,100,duplicate_vs_duplicate,Rubén García,Rubén García,29,31,Spain,...,9,14,13,14,10,NaN,NaN,NaN,NaN,NaN
1717,Rodri,Rodri,Rodri,100,duplicate_vs_duplicate,Rodri,Rodri,27,28,Spain,...,10,10,7,14,8,NaN,NaN,NaN,NaN,NaN
1718,Rodri,Rodri,Rodri,100,duplicate_vs_duplicate,Rodri,Rodri,27,28,Spain,...,10,10,7,14,8,NaN,NaN,NaN,NaN,NaN
1719,D. López,David López,David López,100,duplicate_vs_duplicate,David López,David López,33,34,Spain,...,9,12,7,11,15,NaN,NaN,NaN,NaN,NaN
1720,D. López,Diego López,Diego López,100,duplicate_vs_duplicate,Diego López,Diego López,21,22,Spain,...,14,15,13,14,12,NaN,NaN,NaN,NaN,NaN
1721,V. Junior,Vini Jr.,Vini Jr.,100,unique_vs_duplicate_exact,Vini Jr.,Vini Jr.,22,24,Brazil,...,5,7,7,7,10,NaN,NaN,NaN,NaN,NaN
1722,J. Cruz,Juan Cruz,Juan Cruz,100,duplicate_vs_duplicate,Juan Cruz,Juan Cruz,30,32,Spain,...,14,11,7,11,14,NaN,NaN,NaN,NaN,NaN
1723,J. Cruz,Juan Cruz,Juan Cruz,100,duplicate_vs_duplicate,Juan Cruz,Juan Cruz,30,32,Spain,...,14,11,7,11,14,NaN,NaN,NaN,NaN,NaN


In [322]:
import json

processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

merged_file_path = processed_dir / "merged_players_24_25_final.csv"
merged_df.to_csv(merged_file_path, index=False)

# Excel dosyası açık olabilir, hata verirse sadece CSV kaydettik
try:
    merged_df.to_excel(processed_dir / "merged_players_24_25_final.xlsx", index=False)
    print("Excel file saved successfully")
except PermissionError:
    print("Excel file is open, only CSV saved")

mapping_file_path = processed_dir / "column_mapping_24_25.json"
with open(mapping_file_path, 'w', encoding='utf-8') as f:
    json.dump(column_mapping, f, indent=2, ensure_ascii=False)

print(f"Files saved:")
print(f"- CSV: {merged_file_path}")
print(f"- Mapping: {mapping_file_path}")
print(f"- Merged dataframe shape: {merged_df.shape}")
print(f"- Total matches with age control: {len(merged_df)}")

Excel file saved successfully
Files saved:
- CSV: ..\data\processed\merged_players_24_25_final.csv
- Mapping: ..\data\processed\column_mapping_24_25.json
- Merged dataframe shape: (1764, 109)
- Total matches with age control: 1764


In [323]:
# Antonio Silva'nın duplicate durumunu kontrol edelim
print("Antonio Silva FIFA 24'te:")
antonio_24 = ratings_24_df[ratings_24_df['short_name'].str.contains('Antonio Silva', case=False, na=False)]
print(antonio_24[['short_name', 'transformed_name']].values)

print("\nAntonio Silva FIFA 25'te:")
antonio_25 = ratings_25_df[ratings_25_df['Name'].str.contains('Antonio Silva', case=False, na=False)]
print(antonio_25[['Name', 'transformed_name']].values)

# A. Silva duplicate eşleştirmelerini kontrol edelim
a_silva_duplicate_matches = matches_df[
    (matches_df['name_24'] == 'A. Silva') & (matches_df['match_type'] == 'duplicate_vs_duplicate')
]
print(f"\nA. Silva duplicate matches:")
print(a_silva_duplicate_matches[['original_name_24', 'original_name_25', 'similarity_score', 'match_type']])

Antonio Silva FIFA 24'te:
[]

Antonio Silva FIFA 25'te:
[]

A. Silva duplicate matches:
Empty DataFrame
Columns: [original_name_24, original_name_25, similarity_score, match_type]
Index: []
